## Restart colab session after installation

In [ ]:
!pip install git+https://github.com/Lednik7/CLIP-ONNX.git

In [1]:
%%capture
!wget -c -O CLIP.png https://github.com/openai/CLIP/blob/main/CLIP.png?raw=true

In [12]:
import clip
from PIL import Image

# onnx cannot work with cuda
model, preprocess = clip.load("ViT-B/32", device="cpu", jit=False)
# batch first
image = preprocess(Image.open("CLIP.png")).unsqueeze(0) # [1, 3, 224, 224]
text = clip.tokenize(["a diagram", "a dog", "a cat"]) # [3, 77]

In [17]:
%timeit model(image, text)

1 loop, best of 5: 580 ms per loop


In [ ]:
from clip_onnx import clip_onnx, attention
clip.model.ResidualAttentionBlock.attention = attention

# ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
onnx_model = clip_onnx(model, providers=["CPUExecutionProvider"]) # cpu mode
onnx_model.convert2onnx(image, text, verbose=True)
onnx_model.start_sessions()

In [16]:
image_features = onnx_model.encode_image(image)
text_features = onnx_model.encode_text(text)

logits_per_image, logits_per_text = onnx_model(image, text)
probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)  # prints: [[0.41456965 0.29270944 0.29272085]]

Label probs: [[0.41456965 0.29270944 0.29272085]]


In [9]:
%timeit onnx_model(image, text) # cuda speed up 3x

1 loop, best of 5: 545 ms per loop
